# FCDLツール（横浜ダイアログ用）

In [1]:
### Terminalで: sudo yum install -y ipa-gothic-fonts
!pip install sentence-transformers
!pip install openai==0.28
!pip install pyvis
!pip install pyathena
!pip install --upgrade google-api-python-client

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/86.0 kB ? eta -:--:--
     ------------- ------------------------ 30.7/86.0 kB 640.0 kB/s eta 0:00:01
     ---------------------------------------- 86.0/86.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for torch>=1.6.0 from https://files.pythonhosted.org/packages/e4/ae/2ad8820045b6631965750435f28583e80905b8273d57cf026163b51323ee/torch-2.1.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/f9/e6/3c821e7417acd82df89e39f09156ce80d58817b5b4b1ac5453b522bc5dd4/torchvision-0.16.2-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/977.5 kB ? eta -:--:--
     ------------------------------------- 977.5/977.5 kB 20.6 MB/s eta 0:00:00
   ----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for openai==0.28 from https://files.pythonhosted.org/packages/ae/59/911d6e5f1d7514d79c527067643376cddcf4cb8d1728e599b3b03ab51c69/openai-0.28.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/76.5 kB ? eta -:--:--
   --------------------- ------------------ 41.0/76.5 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 76.5/76.5 kB 1.1 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
      --------------------------------------- 10.2/756.0 kB ? eta -:--:--
     --- --------------------------------- 71.7/756.0 kB 777.7 kB/s eta 0:00:01
     ------------------------------- ------ 624.6/756.0 kB 4.9 MB/s eta 0:00:01
     -------------------------------------- 756.0/756.0 kB 4.8 MB/s eta 0:00:00
  Obtaining dependency information for jsonpickle>=1.4.1 from https://files.pythonhosted.org/packages/d3/25/6e0a450430b7aa194b0f515f64820fc619314faa289458b7dfca4a026114/jsonpickle-3.0.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/40.7 kB ? eta -:--:--
   ---------------------------------------- 40.7/40.7 kB 1.9 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pyathena from https://files.pythonhosted.org/packages/84/3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.5.0 requires botocore<1.29.77,>=1.29.76, but you have botocore 1.34.3 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for google-api-python-client from https://files.pythonhosted.org/packages/62/1b/3ff6bd5f33c1a1780835725014ac480128d2d1e3244b2809275d0fa4f726/google_api_python_client-2.111.0-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/96.9 kB ? eta -:--:--
     ------------ --------------------------- 30.7/96.9 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 96.9/96.9 kB 1.8 MB/s eta 0:00:00
  Obtaining dependency information for google-auth<3.0.0.dev0,>=1.19.0 from https://files.pythonhosted.org/packages/f4/d2/9f6f3b9c0fd486617816cff42e856afea079d0bad99f0e60dc186c76b881/google_auth-2.25.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-httplib2>=0.1.0 from https://files.pythonhosted.org/packages/be/8a/fe34d2f3f9470a27b01c9e76226965863f153d5fbe276f83608562e49c04/google_auth_httplib2-0.2.0-py2.py3-none-

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [11]:
import boto3
import json
import os
import pandas as pd
import datetime
from collections import Counter
import itertools
import re
from scipy import spatial
from pyvis.network import Network
from sentence_transformers import SentenceTransformer, util
import openai
import numpy as np
import pyathena
import pickle
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_distances
from sklearn.cluster import KMeans
from googleapiclient.discovery import build

# APIキーの設定
apis = pd.read_csv("s3://lnext-for-quicksight/keys/apis.csv")
openai.api_key = apis["openai"][0]
API_KEY = apis["googlesearch"][0]
CUSTOM_SEARCH_ENGINE = 'a78a2783d10a44be6'
page_limit = 1
PROMPT_TEXT_KADAI_CORP = '''
    この文章の主題を15文字以上30文字未満で要約してください。
'''

PROMPT_TEXT_FC = '''
    この文章の主題を要約すると次の5個です。それぞれ15文字以上30文字未満で要約してください。
'''

PROMPT_TEXT_OTR = '''
    この文章の主題を要約すると次の3個です。それぞれ15文字以上30文字未満で要約してください。
'''

# Sentence transformersのモデルの読み込み
model = SentenceTransformer('stsb-xlm-r-multilingual')

# アウトプットのトークン数。長い文章を返す場合には大きな数字にする。
OUTPUT_TOKENS = 1000

# distanceの数式の変更
MAX_DISTANCE = 3.0
EMPTY_TEXT = ''
SIMILARITY_UPPER_THRESHOLD = 3.00
SIMILARITY_LOWER_THRESHOLD = 2.65
RESCALED_MAX_SIMILARITY_VALUE = 3.0
RESCALED_MIN_SIMILARITY_VALUE = 0.1

distance_fc_list = []
distance_otr_list = []
distance_fc_and_otr_list = []

def getImageUrl(api_key, cse_key, search_word):
    service = build("customsearch", "v1", developerKey=api_key)
    page_limit = 1
    startIndex = 1
    response = []
    img_list = []

    try:
        response.append(service.cse().list(
            q=search_word,     # Search words
            cx=cse_key,        # custom search engine key
            lr='lang_ja',      # Search language
            num=1,            # Number of images obtained by one request (Max 10)
            start=startIndex,
            searchType='image' # search for images
        ).execute())

        startIndex = response[0].get("queries").get("nextPage")[0].get("startIndex")

    except Exception as e:
        print(e)

    for one_res in range(len(response)):
        if int(response[one_res]["searchInformation"]["totalResults"]) > 0:
            for i in range(len(response[one_res]['items'])):
                img_list.append(response[one_res]['items'][i]['link'])
        else:
            img_list.append("https://1.bp.blogspot.com/-d3vDLBoPktU/WvQHWMBRhII/AAAAAAABL6E/Grg-XGzr9jEODAxkRcbqIXu-mFA9gTp3wCLcBGAs/s800/internet_404_page_not_found.png")

    return img_list[0]

# 結果を取得する関数の定義
def get_results_for_one_text(in_text, prompt):
    response = openai.Completion.create(
        # model="gpt-4",
        model="text-davinci-003",
        prompt=_get_cleaned_text(in_text) + prompt,
        temperature=0, #temperature=0.5,
        max_tokens=OUTPUT_TOKENS,
        top_p=1.0,
        frequency_penalty=0.8,
        presence_penalty=0.0
    )
    return _get_numbering_removed_keyword(
        _get_parsed_result_by_return(
            response["choices"][0]["text"]
        ))


# インプットのテキストを整形する関数の定義。全角スペースを半角スペースに変換。
def _get_cleaned_text(in_text):
    in_text = in_text.replace('　', ' ')
    return in_text

def _get_empty_text_removed_list(in_list):
    return [s for s in in_list if s != EMPTY_TEXT]

def _get_parsed_result_by_return(in_sentence):
    return _get_empty_text_removed_list(in_sentence.split('\n'))

def _get_numbering_removed_keyword(in_keyword_list):
    this_result = []
    for this_keyword in in_keyword_list:
        this_keyword = re.sub(r'^\d+\. ', '', this_keyword)
        this_result.append(re.sub(r'^\d+\.', '', this_keyword))
    return this_result

# 文章ベクトルを取得する関数の定義
def get_sentence_vector(in_sentence):
    return model.encode(in_sentence, convert_to_tensor=False)

# 二つの文章ベクトルの間の距離を取得する関数の定義
def get_similarity(in_vec_a, in_vec_b):
    return MAX_DISTANCE - spatial.distance.cosine(in_vec_a, in_vec_b)

def create_fcdl_network(fc_df, key_pair_df, image_flag, fname, tp, emb_df, network_num, orig_df):
    flag_master = pd.DataFrame({
        "User": ["FC", "DLData", "DLKadai", "DLCorp", "DLHojo", "UserAns"], 
        "Flag": [1, 2, 3, 4, 5, 6], 
        "NodeColor": ["#66c2a5", "#fc8d62", "#8da0cb", "#e78ac3", "#a6d854", "#ffd92f"], 
        "EdgeColor": ["#b3e2cd", "#fdcdac", "#cbd5e8", "#f4cae4", "#e6f5c9", "#fff2ae"]
    })
    
    connect_df1 = fc_df[["Keyword", "UserCluster"]]
    connect_df1.columns = ["key1", "key2"]
    connect_df2 = key_pair_df.sort_values(["similarity"], ascending=False).head(network_num).reset_index(drop=True)[["key1", "key2"]]
    
    if tp == "fcdl":
        fcdl_k1 = []
        fcdl_k2 = []
        for dttp in ["DLData", "DLKadai", "DLCorp", "DLHojo", "UserAns"]:
            for i in range(len(emb_df)):
                fcdl_k1.append(f"FC{str(i)}")
                fcdl_k2.append(f"{dttp}{str(i)}")
        connect_df3 = pd.DataFrame({
            'key1': fcdl_k1, 
            'key2': fcdl_k2
        })
        
        connect_df4 = fc_df[["Keyword", "Corp"]].copy()
        connect_df4 = connect_df4[connect_df4["Keyword"] != ""]
        connect_df4 = connect_df4[connect_df4["Corp"] != ""]
        connect_df4.columns = ["key1", "key2"]
        connect_df5 = orig_df[["Keyword", "User"]].copy()
        connect_df5 = connect_df5[connect_df5["Keyword"] != ""]
        connect_df5 = connect_df5[connect_df5["User"] != ""]
        connect_df5.columns = ["key1", "key2"]
        # connect_df6 = orig_df[orig_df["Answer"] == 1].reset_index(drop=True).copy()
        # connect_df6 = orig_df.copy()
        # connect_df6 = connect_df6[["Keyword", "Cluster"]].copy()
        # connect_df6["Cluster"] = [f"FC{str(clst)}" for clst in connect_df6["Cluster"]]
        # connect_df6 = connect_df6[connect_df6["Keyword"] != ""]
        # connect_df6 = connect_df6[connect_df6["Cluster"] != ""]
        # connect_df6.columns = ["key1", "key2"]
        
        
        connect_df = pd.concat([connect_df1, connect_df2, connect_df3, connect_df4, connect_df5]).dropna().drop_duplicates().reset_index(drop=True)
    else:
        connect_df = pd.concat([connect_df1, connect_df2]).drop_duplicates().reset_index(drop=True)
    
    onlyfc_gp = pd.merge(fc_df[["Keyword", "User"]], flag_master[["User", "Flag"]], on="User", how="left")[["Keyword", "Flag"]]
    onlyfc_gp.columns = ["NodeName", "Flag"]
    
    node_df1 = connect_df[["key1"]]
    node_df1.columns = ["NodeName"]
    node_df2 = connect_df[["key2"]]
    node_df2.columns = ["NodeName"]
    node_df = pd.concat([node_df1, node_df2]).drop_duplicates().reset_index(drop=True)
    node_df["NodeNum"] = [1+i for i in range(len(node_df))]
    node_df = pd.merge(node_df, onlyfc_gp, on="NodeName", how="left")
    node_df["Flag"] = node_df["Flag"].fillna(0)

    if image_flag:
        img_list = []
        for nd_i in range(len(node_df)):
            search_word = node_df["NodeName"][nd_i]
            print(search_word)
            img = getImageUrl(API_KEY, CUSTOM_SEARCH_ENGINE, search_word)
            img_list.append(img)
        node_df['ImageURL'] = img_list
    
    connect_df = pd.merge(connect_df, node_df, left_on="key1", right_on="NodeName", how="left")[["key1", "key2", "NodeNum", "Flag"]]
    connect_df.columns = ["key1", "key2", "NodeNum1", "Flag1"]
    connect_df = pd.merge(connect_df, node_df, left_on="key2", right_on="NodeName", how="left")[["key1", "key2", "NodeNum1", "Flag1", "NodeNum", "Flag"]]
    connect_df.columns = ["key1", "key2", "NodeNum1", "Flag1", "NodeNum2", "Flag2"]
    # connect_df["Flag1"] = [1 if k1[:2] == "FC" else flg for flg, k1 in zip(connect_df["Flag1"], connect_df["key1"])]
    # connect_df["Flag2"] = [1 if k2[:2] == "FC" else flg for flg, k2 in zip(connect_df["Flag2"], connect_df["key2"])]
    connect_df = connect_df.fillna(0)
    connect_df["FCConnect"] = connect_df["Flag1"] * connect_df["Flag2"]

    # ネットワークのインスタンス生成
    network = Network(
        height="1000px",  # デフォルト "500px"
        width="2000px",  # デフォルト "500px"
        notebook=True,  # これをTrueにしておくとjupyter上で結果が見れる
        bgcolor='#ffffff',  # 背景色。デフォルト "#ffffff"
        directed=False,  # Trueにすると有向グラフ。デフォルトはFalseで無向グラフ
    )

    # add_node でノードを追加
    for i in range(len(node_df)):
        nd1_id = int(node_df['NodeNum'][i])
        nd1_name = node_df['NodeName'][i]
        nd1_flag = node_df['Flag'][i]

        if nd1_flag == 1:
            nd1_color = flag_master["EdgeColor"][0]
        elif nd1_flag == 2:
            nd1_color = flag_master["EdgeColor"][1]
        elif nd1_flag == 3:
            nd1_color = flag_master["EdgeColor"][2]
        elif nd1_flag == 4:
            nd1_color = flag_master["EdgeColor"][3]
        elif nd1_flag == 5:
            nd1_color = flag_master["EdgeColor"][4]
        elif nd1_flag == 6:
            nd1_color = flag_master["EdgeColor"][5]
        elif nd1_name[:2] == "FC":
            nd1_color = flag_master["NodeColor"][0]
        elif nd1_name[:6] == "DLData":
            nd1_color = flag_master["NodeColor"][1]
        elif nd1_name[:7] == "DLKadai":
            nd1_color = flag_master["NodeColor"][2]
        elif nd1_name[:6] == "DLCorp":
            nd1_color = flag_master["NodeColor"][3]
        elif nd1_name[:6] == "DLHojo":
            nd1_color = flag_master["NodeColor"][4]
        elif nd1_name[:7] == "UserAns":
            nd1_color = flag_master["NodeColor"][5]
        else:
            nd1_color = "#e5c494"
        
        if image_flag:
            nd1_image = node_df['ImageURL'][i]
            network.add_node(n_id=nd1_id, label=nd1_name, shape='image', image =nd1_image)
        else:
            network.add_node(n_id=nd1_id, label=nd1_name, color=nd1_color)

    for i in range(len(connect_df)):
        nd1_id = int(connect_df['NodeNum1'][i])
        nd2_id = int(connect_df['NodeNum2'][i])
        nd_flag = connect_df['FCConnect'][i]

        if nd_flag == 1:
            edge_color = "#2c7bb6"
        else:
            edge_color = "#1a9641"

        # network.add_edge(nd1_id, nd2_id, color=edge_color, width = edge_width)
        network.add_edge(nd1_id, nd2_id, color=edge_color, width=0.1)
    # 指定したファイル名でHTMLを出力。
    network.show(f"./htmls/fc_{fname}_{tp}.html")

    return 0

def clustering_and_create_network(fc_df, embdata, embkadai, embcorp, embhojo, sentencedata, sentencekadai, sentencecorp, sentencehojo, seg, num_clusters, network_num):
    #k-means法でDLをクラスタリング
    vectors = []
    for i in range(len(fc_df)):
        vectors.append(np.array(fc_df["Vector"][i]))
    norm_vectors = normalize(vectors)
    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    kmeans.fit(norm_vectors)
    fc_df["Cluster"] = kmeans.labels_
    fc_emb_df = fc_df[["Sentence", "Cluster"]].groupby(["Cluster"])["Sentence"].apply('。'.join).reset_index()

    emb_list = []
    for i in range(len(kmeans.cluster_centers_)):
        emb_list.append(model.encode(fc_emb_df["Sentence"][i], convert_to_tensor=True))

    # 入力文と検索対象文のベクトル表現の類似度を計算
    tgt_cluster_list = []
    dl_sentence_list = []
    dl_num_list = []
    dl_corp_list = []
    dl_dtype_list = []
    for emb_i in range(len(emb_list)):
        embedding = emb_list[emb_i]
        
        # data
        scores = util.pytorch_cos_sim(embedding, embdata)
        sorted, indices = scores.sort(descending=True)
        for i in range(5):
            predicted_idx = int(indices[0][i]) # スコアが最大のインデックスの取得
            tgt_cluster_list.append(emb_i)
            dl_sentence_list.append(sentencedata["sentence"][predicted_idx][:50])
            dl_num_list.append(f"DLData,{str(i)}")
            dl_dtype_list.append("DLData")
            dl_corp_list.append(sentencedata["counterpart"][predicted_idx])

        # kadai
        scores = util.pytorch_cos_sim(embedding, embkadai)
        sorted, indices = scores.sort(descending=True)
        for i in range(2):
            predicted_idx = int(indices[0][i]) # スコアが最大のインデックスの取得
            tgt_cluster_list.append(emb_i)
            dl_sentence_list.append(sentencekadai["sentence"][predicted_idx][:50])
            dl_num_list.append(f"DLKadai,{str(i)}")
            dl_dtype_list.append("DLKadai")
            dl_corp_list.append(sentencekadai["counterpart"][predicted_idx])
            
        # corp
        scores = util.pytorch_cos_sim(embedding, embcorp)
        sorted, indices = scores.sort(descending=True)
        for i in range(3):
            predicted_idx = int(indices[0][i]) # スコアが最大のインデックスの取得
            tgt_cluster_list.append(emb_i)
            dl_sentence_list.append(sentencecorp["sentence"][predicted_idx][:50])
            dl_num_list.append(f"DLCorp,{str(i)}")
            dl_dtype_list.append("DLCorp")
            dl_corp_list.append(sentencecorp["counterpart"][predicted_idx])
            
        # hojo
        scores = util.pytorch_cos_sim(embedding, embhojo)
        sorted, indices = scores.sort(descending=True)
        for i in range(2):
            predicted_idx = int(indices[0][i]) # スコアが最大のインデックスの取得
            tgt_cluster_list.append(emb_i)
            dl_sentence_list.append(sentencehojo["sentence"][predicted_idx][:50])
            dl_num_list.append(f"DLHojo,{str(i)}")
            dl_dtype_list.append("DLHojo")
            dl_corp_list.append(sentencehojo["counterpart"][predicted_idx])
            
        # all answer
        fc_flt = fc_df[fc_df["Cluster"] == emb_i].reset_index(drop=True)
        for fc_i in range(len(fc_flt)):
            tgt_cluster_list.append(emb_i)
            dl_sentence_list.append(fc_flt["Keyword"][fc_i])
            dl_num_list.append(f"UserAns,{str(emb_i)}")
            dl_dtype_list.append("UserAns")
            dl_corp_list.append(fc_flt["User"][fc_i])
            
            
    dl_df = pd.DataFrame({
        'Sentence': dl_sentence_list, 
        'User': dl_num_list, 
        'Cluster': tgt_cluster_list, 
        'Corp': dl_corp_list, 
        'DataType': dl_dtype_list
    })
    dl_emb_df = dl_df.copy()

    # 各クラスタからKeyPhraseを取得
    fc_keyword_all = []
    fc_users = []
    fc_cluster_list = []
    fc_vector_list = []
    fc_corp_list = []
    for i in range(len(fc_emb_df)):
        fc_sentence = fc_emb_df["Sentence"][i]
        fc_keyword_list = get_results_for_one_text(fc_sentence[:OUTPUT_TOKENS], PROMPT_TEXT_FC)
        for kw_i in range(len(fc_keyword_list)):
            kw = fc_keyword_list[kw_i].replace("・", "").replace(" ", "").replace("　", "")
            
            # if (len(kw) >= 3) & (len(kw) < 50):
            if (len(kw) >= 3):
                fc_keyword_all.append(kw)
                fc_users.append("FC")
                fc_cluster_list.append(i)
                fc_vector_list.append(get_sentence_vector(kw))
                fc_corp_list.append("")

        flt_dl = dl_emb_df[dl_emb_df["Cluster"] == i].reset_index(drop=True)
        for dl_i in range(len(flt_dl)):
            dl_sentence = flt_dl["Sentence"][dl_i]
            dl_user = flt_dl["DataType"][dl_i]
            dl_corp = flt_dl["Corp"][dl_i]
            if dl_user == "DLData":
                kw = dl_sentence.strip().replace("・", "").replace(" ", "").replace("　", "")
                # if (len(kw) >= 3) & (len(kw) < 50):
                if (len(kw) >= 3):
                    fc_keyword_all.append(kw)
                    fc_users.append(dl_user)
                    fc_cluster_list.append(i)
                    fc_vector_list.append(get_sentence_vector(kw))
                    fc_corp_list.append(dl_corp)
            elif dl_user == "DLKadai":
                dl_keyword_list = get_results_for_one_text(dl_sentence[:OUTPUT_TOKENS], PROMPT_TEXT_KADAI_CORP)
                for kw_i in range(len(dl_keyword_list)):
                    kw = dl_keyword_list[kw_i].replace("・", "").replace(" ", "").replace("　", "")
                    # if (len(kw) >= 3) & (len(kw) < 50):
                    if (len(kw) >= 3):
                        fc_keyword_all.append(kw)
                        fc_users.append(dl_user)
                        fc_cluster_list.append(i)
                        fc_vector_list.append(get_sentence_vector(kw))
                        fc_corp_list.append(dl_corp)
            elif dl_user == "DLCorp":
                dl_keyword_list = get_results_for_one_text(dl_sentence[:OUTPUT_TOKENS], PROMPT_TEXT_KADAI_CORP)
                for kw_i in range(len(dl_keyword_list)):
                    kw = dl_keyword_list[kw_i].replace("・", "").replace(" ", "").replace("　", "")
                    # if (len(kw) >= 3) & (len(kw) < 50):
                    if (len(kw) >= 3):
                        fc_keyword_all.append(kw)
                        fc_users.append(dl_user)
                        fc_cluster_list.append(i)
                        fc_vector_list.append(get_sentence_vector(kw))
                        fc_corp_list.append(dl_corp)
            elif dl_user == "DLHojo":
                kw = dl_sentence.strip().replace("・", "").replace(" ", "").replace("　", "")
                if (len(kw) >= 3):
                    fc_keyword_all.append(kw)
                    fc_users.append(dl_user)
                    fc_cluster_list.append(i)
                    fc_vector_list.append(get_sentence_vector(kw))
                    fc_corp_list.append(dl_corp)
            elif dl_user == "UserAns":
                kw = dl_sentence.strip().replace("・", "").replace(" ", "").replace("　", "")
                # if (len(kw) >= 3) & (len(kw) < 50):
                if (len(kw) >= 3):
                    fc_keyword_all.append(kw)
                    fc_users.append(dl_user)
                    fc_cluster_list.append(i)
                    fc_vector_list.append(get_sentence_vector(kw))
                    fc_corp_list.append(dl_corp)
            else:
                dl_keyword_list = get_results_for_one_text(dl_sentence[:OUTPUT_TOKENS], PROMPT_TEXT_OTR)
                for kw_i in range(len(dl_keyword_list)):
                    kw = dl_keyword_list[kw_i].replace("・", "").replace(" ", "").replace("　", "")
                    # if (len(kw) >= 3) & (len(kw) < 50):
                    if (len(kw) >= 3):
                        fc_keyword_all.append(kw)
                        fc_users.append(dl_user)
                        fc_cluster_list.append(i)
                        fc_vector_list.append(get_sentence_vector(kw))
                        fc_corp_list.append(dl_corp)

    fc_df_base = pd.DataFrame({
        'Keyword': fc_keyword_all, 
        'User': fc_users, 
        'Cluster': fc_cluster_list, 
        'Vector': fc_vector_list, 
        "Corp": fc_corp_list
    })

    vectors = []
    for i in range(len(fc_df_base)):
        vectors.append(np.array(fc_df_base["Vector"][i]))
    norm_vectors = normalize(vectors)
    fc_df_base["NormVector"] = [nv for nv in norm_vectors]
    fc_df_base["UserCluster"] = [f"{usr}{clst}" for usr, clst in zip(fc_df_base["User"], fc_df_base["Cluster"])]
    
    # FC単体情報
    fc_df_onlyfc = fc_df_base[fc_df_base["User"] == "FC"].reset_index(drop=True).copy()
    key_pair_list_onlyfc = list(itertools.combinations(fc_df_onlyfc["Keyword"], 2))
    key1_list = []
    key2_list = []
    sim_list = []
    for i in range(len(key_pair_list_onlyfc)):
        key1 = key_pair_list_onlyfc[i][0]
        key2 = key_pair_list_onlyfc[i][1]
        if fc_df_onlyfc[fc_df_onlyfc["Keyword"] == key1].reset_index(drop=True)["UserCluster"][0] != fc_df_onlyfc[fc_df_onlyfc["Keyword"] == key2].reset_index(drop=True)["UserCluster"][0]:
            vec1 = fc_df_onlyfc[fc_df_onlyfc["Keyword"] == key1].reset_index(drop=True)["NormVector"][0]
            vec2 = fc_df_onlyfc[fc_df_onlyfc["Keyword"] == key2].reset_index(drop=True)["NormVector"][0]
            sim_key1_key2 = get_similarity(vec1, vec2)
            key1_list.append(key1)
            key2_list.append(key2)
            sim_list.append(sim_key1_key2)

    key_pair_df_onlyfc = pd.DataFrame({
        'key1': key1_list, 
        'key2': key2_list, 
        'similarity': sim_list
    })
    print(len(key_pair_df_onlyfc))
    
    # FC&DL両方
    fc_df_all = fc_df_base.copy()
    key_pair_list_all = list(itertools.combinations(fc_df_all["Keyword"], 2))
    key1_list = []
    key2_list = []
    sim_list = []
    for i in range(len(key_pair_list_all)):
        key1 = key_pair_list_all[i][0]
        key2 = key_pair_list_all[i][1]
        if fc_df_all[fc_df_all["Keyword"] == key1].reset_index(drop=True)["UserCluster"][0] != fc_df_all[fc_df_all["Keyword"] == key2].reset_index(drop=True)["UserCluster"][0]:
            vec1 = fc_df_all[fc_df_all["Keyword"] == key1].reset_index(drop=True)["NormVector"][0]
            vec2 = fc_df_all[fc_df_all["Keyword"] == key2].reset_index(drop=True)["NormVector"][0]
            sim_key1_key2 = get_similarity(vec1, vec2)
            key1_list.append(key1)
            key2_list.append(key2)
            sim_list.append(sim_key1_key2)

    key_pair_df_all = pd.DataFrame({
        'key1': key1_list, 
        'key2': key2_list, 
        'similarity': sim_list
    })
    print(len(key_pair_df_all))

    # HTMLを作成
    # create_fcdl_network(fc_df_onlyfc, key_pair_df_onlyfc, False, seg, "onlyfc", fc_emb_df, network_num, fc_df)
    create_fcdl_network(fc_df_all, key_pair_df_all, False, seg, "fcdl", fc_emb_df, network_num, fc_df)
    # create_fcdl_network(fc_df_onlyfc, key_pair_df_onlyfc, True, seg, "onlyfc_img", fc_emb_df, network_num, fc_df)
    # create_fcdl_network(fc_df_all, key_pair_df_all, True, seg, "fcdl_img", fc_emb_df, network_num, fc_df)


# 課題データセットの取得

In [2]:
kadai_df = pd.read_csv("yokohama_kyoso_theme.csv")
kadai_df.columns = ["kid", "theme", "gaiyou", "shokan"]
kadai_df["fulltext"] = [f"{tm}, {gy}" for tm, gy in zip(kadai_df["theme"], kadai_df["gaiyou"])]

sentences_kadai = []
for add_s in kadai_df["fulltext"]:
    sentences_kadai.append(add_s.strip())
    
sentences_kadai_df = pd.DataFrame({
    "sentence": sentences_kadai
})
sentences_kadai_df["counterpart"] = kadai_df["shokan"]

#Compute embeddings
if False:
    print("Start calc embeddings")
    print(datetime.datetime.now())
    embeddings_kadai = model.encode(sentences_kadai, convert_to_tensor=True)
    print(datetime.datetime.now())

    # pickle化してファイルに書き込み
    with open('dataleaves_kadai.pkl', 'wb') as f:
        pickle.dump(embeddings_kadai, f)
else:
    with open('dataleaves_kadai.pkl', 'rb') as f:
        embeddings_kadai = pickle.load(f)

print(len(embeddings_kadai))
print(len(sentences_kadai_df))
sentences_kadai_df.tail()

28
28


,sentence,counterpart
23,"横浜市中央児童相談所の環境改善のための連携について（PDF：307KB）, 横浜市児童相談所...",横浜市こども青少年局中央児童相談所庶務係
24,"ＨＩＶ・エイズ、性感染症検査等の普及啓発（PDF：758KB）, ＨＩＶ／エイズの感染経路は...",横浜市健康福祉局健康安全課結核・エイズ担当
25,「市民と企業によるまちづくり」\r\n協働による地域の課題解決・魅力向上のための施設（ハード...,横浜市都市整備局地域まちづくり課
26,"「横浜消防」のＰＲ・ブランディングと市民の防災・減災意識の啓発（PDF：329KB）, 大地...",横浜市消防局総務部企画課
27,"市民の読書活動の推進のための連携, 読書活動は言葉を学び、感性を磨き、表現力を高め、創造力を...",横浜市教育委員会事務局生涯学習文化財課


# 企業データセットの取得

In [3]:
corp_df = pd.read_csv("yokohama_chiikikouken.csv")
corp_df["fulltext"] = [f"{pr}, {jg}, {tk}" for pr, jg, tk in zip(corp_df["PR"], corp_df["jigyou"], corp_df["torikumi"])]
corp_df["fulltext"] = [st.replace("nan, ", "") for st in corp_df["fulltext"]]
corp_df["fulltext"] = [st.replace(", nan", "") for st in corp_df["fulltext"]]

sentences_corp = []
for add_s in corp_df["fulltext"]:
    sentences_corp.append(add_s.strip())
    
sentences_corp_df = pd.DataFrame({
    "sentence": sentences_corp
})
sentences_corp_df["counterpart"] = [st.strip().replace("株式会社", "").replace("ホールディングス", "") for st in corp_df["corpname"]]

#Compute embeddings
if False:
    print("Start calc embeddings")
    print(datetime.datetime.now())
    embeddings_corp = model.encode(sentences_corp, convert_to_tensor=True)
    print(datetime.datetime.now())

    # pickle化してファイルに書き込み
    with open('dataleaves_corp.pkl', 'wb') as f:
        pickle.dump(embeddings_corp, f)
else:
    with open('dataleaves_corp.pkl', 'rb') as f:
        embeddings_corp = pickle.load(f)

print(len(embeddings_corp))
print(len(sentences_corp_df))
sentences_corp_df.tail()

466
466


,sentence,counterpart
461,横浜市内を中心に公共工事、土木舗装工事を主としています。,和紘建設
462,総合建設業,渡辺組
463,nan,渡辺商事
464,台風・地震・大雨等の天災時にも直ちに出動できる態勢を整えており、横浜市の“災害登録業者”にも...,綿貫建設
465,化粧品の製造卸,ワミレスコスメティックス


# 補助金・助成金データの取得

In [4]:
hojo_yoko = pd.read_csv("yokohama_hojokin_header.csv")
hojo_yoko["fulltext"] = [f"{dh}" for dh in zip(hojo_yoko["datahead"])]
hojo_yoko = hojo_yoko[["cid", "fulltext", "institute", "datahead"]].copy()
hojo_yoko.columns = ["data_number", "fulltext", "institute", "datahead"]

hojo_yoko_desc = pd.read_csv("yokohama_hojokin_name.csv")
hojo_yoko_desc["fulltext"] = [f"{dh} {dn} {obj} {kh}" for dh, dn, obj, kh in zip(hojo_yoko_desc["datahead"], hojo_yoko_desc["dataname"], hojo_yoko_desc["object"], hojo_yoko_desc["keihi"])]
hojo_yoko_desc = hojo_yoko_desc[["cid", "fulltext", "institute", "datahead"]].copy()
hojo_yoko_desc.columns = ["data_number", "fulltext", "institute", "datahead"]

sentences_hojo = []
for add_s in hojo_yoko["fulltext"]:
    sentences_hojo.append(add_s.strip())
sentences_hojo_df = pd.DataFrame({
    "sentence": sentences_hojo
})
sentences_hojo_df["counterpart"] = [st.strip() for st in hojo_yoko["institute"]]

sentences_hojo_desc = []
for add_s in hojo_yoko_desc["fulltext"]:
    sentences_hojo_desc.append(add_s.strip())
sentences_hojo_desc_df = pd.DataFrame({
    "sentence": sentences_hojo_desc
})
sentences_hojo_desc_df["counterpart"] = [st.strip() for st in hojo_yoko_desc["institute"]]

#Compute embeddings
if False: 
# if True:
    print("Start calc embeddings")
    print(datetime.datetime.now())
    embeddings_hojo = model.encode(sentences_hojo, convert_to_tensor=True)
    embeddings_hojo_desc = model.encode(sentences_hojo_desc, convert_to_tensor=True)
    print(datetime.datetime.now())

    # pickle化してファイルに書き込み
    with open('dataleaves_hojo.pkl', 'wb') as f:
        pickle.dump(embeddings_hojo, f)
    with open('dataleaves_hojo_desc.pkl', 'wb') as f:
        pickle.dump(embeddings_hojo_desc, f)
else:
    with open('dataleaves_hojo.pkl', 'rb') as f:
        embeddings_hojo = pickle.load(f)    
    with open('dataleaves_hojo_desc.pkl', 'rb') as f:
        embeddings_hojo_desc = pickle.load(f)
        
sentences_hojo_df["sentence"] = hojo_yoko["datahead"]
sentences_hojo_desc_df["sentence"] = hojo_yoko_desc["datahead"]
print(len(embeddings_hojo))
print(len(sentences_hojo_df))
print(hojo_yoko.tail())
print(len(embeddings_hojo_desc))
print(len(sentences_hojo_desc_df))
print(hojo_yoko_desc.tail())


79
79
    data_number                             fulltext       institute  \
74           75  ('横浜市：2020年度 U39アーティスト・フェローシップ助成',)     横浜市芸術文化振興財団   
75           76        ('横浜市：クリエイティブ・インクルージョン活動助成',)     横浜市芸術文化振興財団   
76           77                ('LIP. 横浜 トライアル助成金',)  木原記念横浜生命科学振興財団   
77           78              ('「横浜市被災中小企業復旧支援補助金」',)             横浜市   
78           79               ('横浜市 小規模事業者設備投資助成金',)             横浜市   

                          datahead  
74  横浜市：2020年度 U39アーティスト・フェローシップ助成  
75        横浜市：クリエイティブ・インクルージョン活動助成  
76                LIP. 横浜 トライアル助成金  
77              「横浜市被災中小企業復旧支援補助金」  
78               横浜市 小規模事業者設備投資助成金  
79
79
    data_number                                           fulltext  \
74           75  横浜市：2020年度 U39アーティスト・フェローシップ助成 横浜市：2020年度 U39ア...   
75           76  横浜市：クリエイティブ・インクルージョン活動助成 横浜市：クリエイティブ・インクルージョン活...   
76           77  LIP. 横浜 トライアル助成金 LIP. 横浜 トライアル助成金 健康・医療分野での試作品...   
77           78  「横浜市被災中小企業復旧支援補助金」 「横浜市被災中小企業復旧支援補助金

# データジャケットの内容をベクトルとして保存

In [5]:
if False:
    dls = pd.read_csv("datajacket.csv", usecols=["ID", "title", "outline", "collecting_cost", "sharing_policy", "type", "variable", "analysis", "outcome", "anticipation", "comments", "wanted"])
    dls["fulltext"] = [f"{ttl}, {ol}, {v}" for ttl, ol, v in zip(dls["title"], dls["outline"], dls["variable"])]

    sentences = []
    for add_s in dls["fulltext"]:
        sentences.append(add_s.strip())

    #Compute embeddings
    if False:
        print("Start calc embeddings")
        print(datetime.datetime.now())
        embeddings = model.encode(sentences, convert_to_tensor=True)
        print(datetime.datetime.now())

        # pickle化してファイルに書き込み
        with open('dataleaves.pkl', 'wb') as f:
            pickle.dump(embeddings, f)
    else:
        with open('dataleaves.pkl', 'rb') as f:
            embeddings = pickle.load(f)
else:
    dls_yoko = pd.read_csv("yokohama_opendata_header.csv")
    dls_yoko["fulltext"] = [f"{dh}" for dh in zip(dls_yoko["datahead"])]
    dls_yoko = dls_yoko[["cid", "fulltext", "creator"]].copy()
    dls_yoko.columns = ["data_number", "fulltext", "creator"]

    dls_yoko_desc = pd.read_csv("yokohama_opendata_name.csv")
    dls_yoko_desc["fulltext"] = [f"{dn}" for dn in zip(dls_yoko_desc["dataname"])]
    dls_yoko_desc = dls_yoko_desc[["cid", "fulltext", "creator"]].copy()
    dls_yoko_desc.columns = ["data_number", "fulltext", "creator"]

    sentences_yoko = []
    for add_s in dls_yoko["fulltext"]:
        sentences_yoko.append(add_s.strip())
    sentences_yoko_df = pd.DataFrame({
        "sentence": sentences_yoko
    })
    sentences_yoko_df["counterpart"] = [st.strip() for st in dls_yoko["creator"]]

    sentences_yoko_desc = []
    for add_s in dls_yoko_desc["fulltext"]:
        sentences_yoko_desc.append(add_s.strip())
    sentences_yoko_desc_df = pd.DataFrame({
        "sentence": sentences_yoko_desc
    })
    sentences_yoko_desc_df["counterpart"] = [st.strip() for st in dls_yoko_desc["creator"]]

    #Compute embeddings
    # if False: 
    if False:
        print("Start calc embeddings")
        print(datetime.datetime.now())
        embeddings_yoko = model.encode(sentences_yoko, convert_to_tensor=True)
        embeddings_yoko_desc = model.encode(sentences_yoko_desc, convert_to_tensor=True)
        print(datetime.datetime.now())

        # pickle化してファイルに書き込み
        with open('dataleaves_yoko.pkl', 'wb') as f:
            pickle.dump(embeddings_yoko, f)
        with open('dataleaves_yoko_desc.pkl', 'wb') as f:
            pickle.dump(embeddings_yoko_desc, f)
    else:
        with open('dataleaves_yoko.pkl', 'rb') as f:
            embeddings_yoko = pickle.load(f)    
        with open('dataleaves_yoko_desc.pkl', 'rb') as f:
            embeddings_yoko_desc = pickle.load(f)
    print(len(embeddings_yoko))
    print(len(sentences_yoko_df))
    print(dls_yoko.tail())
    print(len(embeddings_yoko_desc))
    print(len(sentences_yoko_desc_df))
    print(dls_yoko_desc.tail())

544
544
     data_number                            fulltext creator
539          540   ('   平成２９年１月 市場月報       XLS   ',)   運営調整課
540          541  ('   平成２８年１２月 市場月報       XLS   ',)   運営調整課
541          542  ('   平成２８年１０月 市場月報       XLS   ',)   運営調整課
542          543   ('   平成２８年９月 市場月報       XLS   ',)   運営調整課
543          544   ('   平成２８年８月 市場月報       XLS   ',)   運営調整課
14208
14208
       data_number                                           fulltext creator
14203        14204  ('   平成２８年８月 市場月報       XLS          鳥卵部\u3000...   運営調整課
14204        14205  ('   平成２８年８月 市場月報       XLS          鳥卵部\u3000...   運営調整課
14205        14206  ('   平成２８年８月 市場月報       XLS          鳥卵部\u3000...   運営調整課
14206        14207  ('   平成２８年８月 市場月報       XLS          鳥卵部\u3000...   運営調整課
14207        14208  ('   平成２８年８月 市場月報       XLS          食肉部\u3000...   運営調整課


# FC登録

In [6]:
qst_goal = f"""
    みなさんが創りたい未来の横浜 、果たしたい役割、 コンソーシアムとしてやりたいことについてお聞かせください。
"""

qst_list = [
    "Qst,（１）あなたが創りたい未来の横浜とは、どのようなものですか？,自由コメント", 
    "Qst,（２）創りたい未来の実現のため、あなたはコンソーシアム内でどのような役割を果たしたいですか？,自由コメント", 
    "Qst,（３）その役割が果たせたかどうかを、どのような指標で測ることができると思いますか？,自由コメント", 
    "Qst,（４）今後に向け、こういうことが出来たらいいな、やってみたいな、という事業あるいは会議体等があれば自由にご回答ください。,自由コメント"
]
qst_list

['Qst,（１）あなたが創りたい未来の横浜とは、どのようなものですか？,自由コメント',
 'Qst,（２）創りたい未来の実現のため、あなたはコンソーシアム内でどのような役割を果たしたいですか？,自由コメント',
 'Qst,（３）その役割が果たせたかどうかを、どのような指標で測ることができると思いますか？,自由コメント',
 'Qst,（４）今後に向け、こういうことが出来たらいいな、やってみたいな、という事業あるいは会議体等があれば自由にご回答ください。,自由コメント']

In [7]:
if False:
    conn = pyathena.connect(
        s3_staging_dir="s3://lnext-for-quicksight/athena-cash/",
        region_name="ap-northeast-1"
    )

    broker = 'kuma'
    event = 'kuma_20231117kuma'

    # sql = f"SELECT * FROM AwsDataCatalog.lnext_sandbox.t_tsugou_kpi2 WHERE broker = '{broker}' AND event = '{event}' order by answer_time desc;"
    sql = f"SELECT * FROM AwsDataCatalog.lnext_sandbox.t_tsugou_kpi2 WHERE broker = '{broker}' order by answer_time desc;"
    opt_df = pd.read_sql(sql, conn)
    print(len(opt_df))
    conn.close()
else:
    opt_df = pd.read_csv("みんなで創りたい未来の横浜（回答） - フォームの回答 1.csv")
    opt_df.columns = ["tstamp", "mail", "ans1", "ans2", "ans3", "ans4", "email"]
opt_df



,tstamp,mail,ans1,ans2,ans3,ans4,email
0,2023/10/22 10:17:43,NaN,横浜を、地域や社会のためになる活動をやりたいと思った人が、自由に相談し、仲間を見つけ、支援者...,事業提案者の理念、チーム、スキル、事業の収益可能性、支援後の効果検証およびモニタリング等の各...,事業設立数、黒字事業数（含む社会的インパクト）、黒字額（含む社会的インパクト）、KPI達成進...,皆さんがやりたいこと、やってみたいことを自由に投稿頂き、あれこれ議論できるようなアイデアサン...,TA前川
1,2023/10/22 15:06:57,NaN,市民、企業、行政が一体となり、高度な自治を実現している街,行政と企業の対話によるオープンイノベーションの成功事例の開拓と、キャリア教育の一環としての子...,行政と企業の共創による事業取り組み数・売上伸び率・行政の生産性向上指数など、小中高生の自己有...,ケースメソッドによる共創創発ワークショプ、子どもと横浜型地域貢献企業が一緒に参加する共創ダイアログ,emo
2,2023/10/22 19:49:45,NaN,誰もが自分らしさを活かし、自然や人と関わりながら楽しく優しく共生している、コンヴィヴィアルなまち,情報発信（市外・海外への発信も含めて）。横浜市内各地で展開されている取り組みの価値を可視化し...,記事数、Webサイトへの訪問者数、記事によって感謝された数,「循環経済とケア」をテーマとするイベントや事業,Yu
3,2023/10/23 8:36:28,NaN,自然と調和しながら、そこに暮らす人が「生きがい」「やりがい」を感じながら、フォーマルなサービ...,団地という多種多様な人々が暮らす場で、住民の声を聞き、それを行政や企業と共有しながら、実走で...,そこに暮らす人々の定量的な幸福度の調査を継続して行いデータ化する。,今、バラバラに動いている様々な取り組みやプロジェクトを点から面に変えていきたいと思います。,一般社団法人団地暮らしの共創 小柴健一
4,2023/10/23 10:14:42,NaN,活力ある企業活動が優れた雇用環境を市内外の人に提供し、産学官による先進的な取組があふれているまち,行政と企業（と大学）が活発なオープンイノベーションに取り組める場の提供と投資の蓄積,オープンイノベーションの件数と企業の投資額や雇用の実績（報告ベースにはなりますが）,産学官のオープンイノベーションの場は既にあるので、そこと共創コンソーシアムとの連携の場、さら...,きんちゃん
5,2023/10/23 11:16:24,NaN,目の前の課題解決に忙しくするのではなく、先進的なことや創造的なことに人々の時間やお金を割くこ...,20代という立場から、自分の持っている価値観や希望、理想を積極的に発信する。,市民意識調査（未来に対する希望・不安を問う設問の回答）、選挙の投票率、人口増加数、市内の世帯...,NaN,Circular Yokohama（ハーチ株式会社）
6,2023/10/24 8:56:06,NaN,良き新らしきものが良き古きものから自然に成長してくるような状況,良き新しきもの、良き古きものを探す遊歩者、それらを繋ぐ篩(ふるい)としての役割,創造的な勉強会の数、創造的なまちあるきの実施回数、継続的な対話の場の質と数,いろんな局の守備範囲をまたいだ継続的な勉強会。具体的なフィールドをみなで観て考える会。市役所...,藤原徹平
7,2023/10/24 11:48:30,NaN,100年後の未来を見据え、子供達の礎となる利他の精神と農福スポーツ連携したまち。平時でも災害...,新しい事業を創出し、それを見える化しながら認知拡大、資金調達、事業支援を持続的に実施する。（...,コンソーシアムの事業として、竹山団地やすすきの団地で生まれた事業が基軸となり、他の地域で事業...,企業はふるさと納税獲得に向けた取り組みを団地をキーワードにタブロイド紙を作成しています。行政...,NPO法人KUSC・神奈川大学サッカー部藤森
8,2023/10/24 16:52:30,NaN,自立型経済都市として自立し、愛を持って人や物とか関わりを持てる街,共創案件のブリッジ人材,自立事業としての案件成功数、経済的（雇用率等含む）な結果,ポケモンや、ガンダム等のインパクトのある事業（関係人口の増加含む）,コミュニティデザイン・ラボ 町山
9,2023/10/24 16:56:37,NaN,18区それぞれに特徴があるので、中区、西区の中心だけでなく、その彩り豊かな特徴を出し合えるまち,それぞれに特徴を活かしあえる情報発信基地,Well-being指標で図る。市民総幸福量の創設など。,人口減少社会でも、明るく仲良く、行ってみたい、住んでみたいまちとなるためにも、みんながまちづ...,とつかリビングラボ


In [8]:
ans_cols_df = pd.DataFrame({"Col": opt_df.columns})
ans_cols_df["flag"] = [1 if (ch[:3] == "ans") & (len(ch) < 10) else 0 for ch in ans_cols_df["Col"]]
ans_cols_df = ans_cols_df[ans_cols_df["flag"] == 1].reset_index(drop=True)
ans_max = len(ans_cols_df)
ans_max

4

In [9]:
fc_sentence_list = []
fc_vector_list = []
fc_user_list = []
fc_ans_list =[]
fc_keyword_list = []
for user_i in range(len(opt_df)):
    print(f"Now calculating {str(user_i+1)} of {str(len(opt_df))}'s data...")
    user = opt_df["email"][user_i]
    for ans_i in range(ans_max):
        fc_sentence = opt_df[f"ans{str(ans_i+1)}"][user_i]
        if type(fc_sentence) == float:
            fc_sentence = ""
        else:
            fc_sentence = fc_sentence.strip()
            
        if len(fc_sentence) >= 3:
            fc_sentence_list.append(fc_sentence)
            fc_vector_list.append(get_sentence_vector(fc_sentence))
            fc_user_list.append(user)
            fc_ans_list.append(ans_i+1)
            
            if len(fc_sentence) < 30:
                fc_keyword_list.append(fc_sentence)
            else:
                fc_keyword = get_results_for_one_text(fc_sentence[:OUTPUT_TOKENS], PROMPT_TEXT_KADAI_CORP)
                fc_keyword_list.append(fc_keyword[0].replace("・", "").replace(" ", "").replace("　", "").replace("「", "").replace("」", ""))

fc_all = pd.DataFrame({
    'Sentence': fc_sentence_list, 
    'Vector': fc_vector_list, 
    'User': fc_user_list, 
    'Answer': fc_ans_list, 
    'Keyword': fc_keyword_list
})


user_all = fc_all["User"].unique()
answer_all = fc_all["Answer"].unique()
fc_all

Now calculating 1 of 12's data...
Now calculating 2 of 12's data...
Now calculating 3 of 12's data...
Now calculating 4 of 12's data...
Now calculating 5 of 12's data...
Now calculating 6 of 12's data...
Now calculating 7 of 12's data...
Now calculating 8 of 12's data...
Now calculating 9 of 12's data...
Now calculating 10 of 12's data...
Now calculating 11 of 12's data...
Now calculating 12 of 12's data...


,Sentence,Vector,User,Answer,Keyword
0,横浜を、地域や社会のためになる活動をやりたいと思った人が、自由に相談し、仲間を見つけ、支援者...,"[0.15350434, -0.011069527, 0.36629382, -0.2243...",TA前川,1,横浜をチャンスある街にする
1,事業提案者の理念、チーム、スキル、事業の収益可能性、支援後の効果検証およびモニタリング等の各...,"[-0.01600974, 0.96617544, 1.1718636, -0.725385...",TA前川,2,事業評価手法の開発運用
2,事業設立数、黒字事業数（含む社会的インパクト）、黒字額（含む社会的インパクト）、KPI達成進...,"[0.21427114, 0.2961616, 0.11891878, -0.4338636...",TA前川,3,社会的インパクトを含む黒字事業
3,皆さんがやりたいこと、やってみたいことを自由に投稿頂き、あれこれ議論できるようなアイデアサン...,"[-0.015601583, 0.3038959, 0.580638, 0.07355337...",TA前川,4,アイデアサンドボックスの構築
4,市民、企業、行政が一体となり、高度な自治を実現している街,"[0.2688053, 0.052769035, 0.8365535, -0.1892348...",emo,1,市民、企業、行政が一体となり、高度な自治を実現している街
5,行政と企業の対話によるオープンイノベーションの成功事例の開拓と、キャリア教育の一環としての子...,"[-0.29456797, 0.026520735, 0.7673636, 0.089263...",emo,2,オープンイノベーションと子ども参加
6,行政と企業の共創による事業取り組み数・売上伸び率・行政の生産性向上指数など、小中高生の自己有...,"[-0.10903608, 0.5291753, 0.030806696, -0.39204...",emo,3,行政企業の共創による成果
7,ケースメソッドによる共創創発ワークショプ、子どもと横浜型地域貢献企業が一緒に参加する共創ダイアログ,"[0.18595317, -0.056578077, 0.451056, 0.0625802...",emo,4,子どもと企業の共創ワークショップ
8,誰もが自分らしさを活かし、自然や人と関わりながら楽しく優しく共生している、コンヴィヴィアルなまち,"[-0.38173732, -0.06489463, 1.0201044, 0.250125...",Yu,1,コンヴィヴィアルな共生まち
9,情報発信（市外・海外への発信も含めて）。横浜市内各地で展開されている取り組みの価値を可視化し...,"[0.23779905, -0.11628204, 0.6812859, -0.343407...",Yu,2,横浜市の取り組みを発信する


In [12]:
# clustering_and_create_network(fc_all, embeddings_kuma_add, sentences_kuma_add, "kumaadd", 5, 25)
clustering_and_create_network(fc_all, embeddings_yoko_desc, embeddings_kadai, embeddings_corp, embeddings_hojo_desc, sentences_yoko_desc_df, sentences_kadai_df, sentences_corp_df, sentences_hojo_desc_df, "yoko", 5, 25)
# clustering_and_create_network(fc_all, embeddings_yoko_desc, sentences_yoko_desc, "yoko_desc", 5, 25)
# clustering_and_create_network(fc_all, embeddings, sentences, "all", 5, 25)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


250
8110
./htmls/fc_yoko_fcdl.html


In [12]:
if False:
    for ans in answer_all:
        fc_df = fc_all[fc_all["Answer"] == ans].reset_index(drop=True).copy()
        clustering_and_create_network(fc_df, f"ans{ans}", 4, 25)

if False:
    for user in user_all:
        fc_df = fc_all[fc_all["User"] == user].reset_index(drop=True).copy()
        clustering_and_create_network(fc_df, f"kpi-user-{user}", 4, 25)
    
